In [1]:
%load_ext autoreload
%autoreload 2

# Pair

### Search for some products from which to build pairs and stacks of pairs

In [2]:
import asf_search as asf

asf.constants.INTERNAL.CMR_TIMEOUT = 90

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
# results = asf.product_search('S1_181296_IW1_20220219T125501_VV_10AF-BURST')
reference = results[0]

args = asf.ASFSearchOptions(
    **{"start": '2022-02-10', "end": '2022-07-01'}
)
s = reference.stack(args)

### Create a Pair object from 2 products

In [3]:
pair = asf.Pair(reference, s[2])

print(f"pair.ref.properties['sceneName']: {pair.ref.properties['sceneName']}")
print(f"pair.ref_date: {pair.ref_date}\n")

print(f"pair.sec.properties['sceneName']: {pair.sec.properties['sceneName']}")
print(f"pair.sec_date: {pair.sec_date}\n")

print(f"pair.temporal: {pair.temporal}")
print(f"pair.perpendicular: {pair.perpendicular}")

pair.ref.properties['sceneName']: S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252
pair.ref_date: 2022-02-15

pair.sec.properties['sceneName']: S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E
pair.sec_date: 2022-03-11

pair.temporal: 24 days, 0:00:00
pair.perpendicular: -73


### Check the estimated coherence of the pair

In [4]:
pair.estimate_s1_mean_coherence()

16.952704858593183

### Create a new pair with a long temporal baseline

In [5]:
long_pair = pair = asf.Pair(reference, s[10])
print(f"long_pair.temporal: {long_pair.temporal}")

long_pair.temporal: 120 days, 0:00:00


### Since the temporal baseline is greater than 48, an exception is raised when checking coherence

In [6]:
long_pair.estimate_s1_mean_coherence()

Exception: Coherence dataset includes temporal baselines up to 48 days.
            Temporal baseline: 120 days

### Demonstrate pair equivalence

Two pairs are equal if they both share the same reference and secondary dates.

In [ ]:
from copy import deepcopy

a = deepcopy(pair)
b = deepcopy(pair)
c = asf.Pair(reference, s[1])
print(f"a.ref_date: {a.ref_date}, a.sec_date: {a.sec_date}")
print(f"b.ref_date: {b.ref_date}, b.sec_date: {b.sec_date}")
print(f"c.ref_date: {c.ref_date}, c.sec_date: {c.sec_date}")
print(f"a == b: {a == b}")
print(f"a == c: {a == c}")

# Stack

### Create a Stack from the reference scene object instantiated above

Stack accepts ASFSearchOptions, which are used in a stack_from_product search

In [7]:
args = asf.ASFSearchOptions(
    **{"start": '2022-01-01', "end": '2022-03-02'}
)

stack = asf.Stack(reference, args)
stack.full_stack

{(datetime.date(2022, 1, 10),
  datetime.date(2022, 1, 22)): Pair(2022-01-10, 2022-01-22),
 (datetime.date(2022, 1, 10),
  datetime.date(2022, 2, 3)): Pair(2022-01-10, 2022-02-03),
 (datetime.date(2022, 1, 10),
  datetime.date(2022, 2, 15)): Pair(2022-01-10, 2022-02-15),
 (datetime.date(2022, 1, 10),
  datetime.date(2022, 2, 27)): Pair(2022-01-10, 2022-02-27),
 (datetime.date(2022, 1, 22),
  datetime.date(2022, 2, 3)): Pair(2022-01-22, 2022-02-03),
 (datetime.date(2022, 1, 22),
  datetime.date(2022, 2, 15)): Pair(2022-01-22, 2022-02-15),
 (datetime.date(2022, 1, 22),
  datetime.date(2022, 2, 27)): Pair(2022-01-22, 2022-02-27),
 (datetime.date(2022, 2, 3),
  datetime.date(2022, 2, 15)): Pair(2022-02-03, 2022-02-15),
 (datetime.date(2022, 2, 3),
  datetime.date(2022, 2, 27)): Pair(2022-02-03, 2022-02-27),
 (datetime.date(2022, 2, 15),
  datetime.date(2022, 2, 27)): Pair(2022-02-15, 2022-02-27)}

### Remove pairs from the full stack, identifying them by ref and sec dates

Both full_stack and subset_stack contain the same Pair objects, so keeping both stacks does not eat up a ton of extra memory

In [8]:
import pandas as pd
import datetime

stack.remove_pairs([
    (datetime.date(2022, 1, 10), datetime.date(2022, 1, 22)), # datetime.date
    (pd.to_datetime("2022-1-10"), pd.to_datetime("2022- 2-3")), # pandas.Timestamp
    ("2022-01-10", "2022-02-15") # isoformat string
])

stack.remove_list

[(datetime.date(2022, 1, 10), datetime.date(2022, 1, 22)),
 (datetime.date(2022, 1, 10), datetime.date(2022, 2, 3)),
 (datetime.date(2022, 1, 10), datetime.date(2022, 2, 15))]

### Look at subset_stack, after having removed some pairs from full_stack

In [9]:
stack.subset_stack

{(datetime.date(2022, 1, 10),
  datetime.date(2022, 2, 27)): Pair(2022-01-10, 2022-02-27),
 (datetime.date(2022, 1, 22),
  datetime.date(2022, 2, 3)): Pair(2022-01-22, 2022-02-03),
 (datetime.date(2022, 1, 22),
  datetime.date(2022, 2, 15)): Pair(2022-01-22, 2022-02-15),
 (datetime.date(2022, 1, 22),
  datetime.date(2022, 2, 27)): Pair(2022-01-22, 2022-02-27),
 (datetime.date(2022, 2, 3),
  datetime.date(2022, 2, 15)): Pair(2022-02-03, 2022-02-15),
 (datetime.date(2022, 2, 3),
  datetime.date(2022, 2, 27)): Pair(2022-02-03, 2022-02-27),
 (datetime.date(2022, 2, 15),
  datetime.date(2022, 2, 27)): Pair(2022-02-15, 2022-02-27)}

### Look at connected_substacks

If subset_stack is disconnected, its component stacks are stored in connected_substacks.

Since we have not yet done anything to disconnect the subset_stack, this list contains only one item.

When we use the Network class to make SBAS stacks, we may have multiple connected substacks.



In [10]:
stack.connected_substacks

[{(datetime.date(2022, 1, 10),
   datetime.date(2022, 2, 27)): Pair(2022-01-10, 2022-02-27),
  (datetime.date(2022, 1, 22),
   datetime.date(2022, 2, 27)): Pair(2022-01-22, 2022-02-27),
  (datetime.date(2022, 2, 3),
   datetime.date(2022, 2, 27)): Pair(2022-02-03, 2022-02-27),
  (datetime.date(2022, 2, 15),
   datetime.date(2022, 2, 27)): Pair(2022-02-15, 2022-02-27),
  (datetime.date(2022, 1, 22),
   datetime.date(2022, 2, 3)): Pair(2022-01-22, 2022-02-03),
  (datetime.date(2022, 1, 22),
   datetime.date(2022, 2, 15)): Pair(2022-01-22, 2022-02-15),
  (datetime.date(2022, 2, 3),
   datetime.date(2022, 2, 15)): Pair(2022-02-03, 2022-02-15)}]

## Network

Create a Network from the same reference scene. 

Attempts to create a connected, seasonal SBAS stack based on:
- georeference scene
- season
- start date
- end date
- perpendicular baseline
- temporal baseline
  

In [11]:
import pandas as pd
from datetime import datetime

def get_julian_season(self) -> tuple[int,int]:
    season_start_ts = pd.Timestamp(
        datetime.strptime(f"{season[0]}-0001", "%m-%d-%Y"), tz="UTC"
        )
    season_start_day = season_start_ts.timetuple().tm_yday
    season_end_ts = pd.Timestamp(
        datetime.strptime(f"{season[1]}-0001", "%m-%d-%Y"), tz="UTC"
    )
    season_end_day = season_end_ts.timetuple().tm_yday
    return (season_start_day, season_end_day)

season = ("1-1", "6-25")

opts = asf.ASFSearchOptions(
    **{
        "start": '2021-01-01', 
        "end": '2023-10-02', 
        "season": get_julian_season(season)
        }
)

network = asf.Network(
    reference,
    perp_baseline=200, 
    temporal_baseline=12,
    bridge_target_date='3-1',
    opts=opts)

print(f"len(network.full_stack): {len(network.full_stack)}")
print(f"len(network.remove_list): {len(network.remove_list)}")
print(f"len(network.subset_stack): {len(network.subset_stack)}")
print(f"len(network.connected_substacks): {len(network.connected_substacks)}")

len(network.full_stack): 946
len(network.remove_list): 899
len(network.subset_stack): 47
len(network.connected_substacks): 2


In [12]:
network.plot()

### Create an updated network with shorter baselines

In [13]:
network = asf.Network(
    reference,
    perp_baseline=50, 
    temporal_baseline=12,
    bridge_target_date='3-1',
    opts=opts)

network.plot()

### Create an updated network with longer baselines

In [14]:
network = asf.Network(
    reference,
    perp_baseline=200, 
    temporal_baseline=24,
    bridge_target_date='3-1',
    opts=opts)

network.plot()

### Plot full_stack

You can pass an any member stack (or list of member stacks) to the plot function.

In [15]:
network.plot(stack_dict=network.full_stack)

### Call `network.get_insar_pair_ids()` on the largest connected substack

Provides the data structure you want when ordering GAMMA or single-burst interferograms from HyP3

In [16]:
network.get_insar_pair_ids()

[('S1A_IW_SLC__1SDV_20210103T225115_20210103T225142_035980_043712_1F5A',
  'S1A_IW_SLC__1SDV_20210115T225115_20210115T225142_036155_043D36_8F80'),
 ('S1A_IW_SLC__1SDV_20210103T225115_20210103T225142_035980_043712_1F5A',
  'S1A_IW_SLC__1SDV_20210127T225114_20210127T225141_036330_044350_4CA8'),
 ('S1A_IW_SLC__1SDV_20210115T225115_20210115T225142_036155_043D36_8F80',
  'S1A_IW_SLC__1SDV_20210127T225114_20210127T225141_036330_044350_4CA8'),
 ('S1A_IW_SLC__1SDV_20210115T225115_20210115T225142_036155_043D36_8F80',
  'S1A_IW_SLC__1SDV_20210208T225114_20210208T225141_036505_04495F_4461'),
 ('S1A_IW_SLC__1SDV_20210127T225114_20210127T225141_036330_044350_4CA8',
  'S1A_IW_SLC__1SDV_20210208T225114_20210208T225141_036505_04495F_4461'),
 ('S1A_IW_SLC__1SDV_20210127T225114_20210127T225141_036330_044350_4CA8',
  'S1A_IW_SLC__1SDV_20210220T225113_20210220T225140_036680_044F7A_D4DB'),
 ('S1A_IW_SLC__1SDV_20210208T225114_20210208T225141_036505_04495F_4461',
  'S1A_IW_SLC__1SDV_20210220T225113_20210220T

### `network.get_insar_pair_ids()` defaults to using the largest connected substack, but you can pass it any member stack

In [17]:
print(len(network.get_insar_pair_ids()))
print(len(network.get_insar_pair_ids(network.full_stack)))

108
946
